In [431]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost 

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 100)

from xgboost import XGBRegressor
from sklearn.linear_model import LassoLars
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import TweedieRegressor

from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer

from prepare import x_y_split, rmse, select_kbest, rfe

In [332]:
df = pd.read_csv('season.csv',index_col=0)

seasons = range(2016,2023)
positions = ['QB','RB','WR','TE']
add = []

for s in seasons:
    for p in positions:
        test = df[(df.year==s)&(df.pos==p)].sort_values('avg_draft_pos_ppr').reset_index(drop=True)
        test['adp_by_pos'] = test.index+1
        
        test = test[(test.year==s)&(test.pos==p)].sort_values('ppr_pts',ascending=False).reset_index(drop=True)
        test['pos_rank'] = test.index+1
        
        add.append(test)

df = pd.concat(add).reset_index(drop=True)

for i in range(0,len(df.index)):
    if df.loc[i,'pos'] =='QB' or df.loc[i,'pos'] =='TE':
        
        if df.loc[i, 'adp_by_pos'] <= 3:
            if df.loc[i,'pos_rank'] <= 3:
                df.loc[i,'success'] = 1
            else:
                df.loc[i, 'success'] = 0
        else:
            if df.loc[i,'pos_rank']<=12:
                df.loc[i,'success'] = 1
            else:
                df.loc[i, 'success'] = 0

    else:
        if df.loc[i, 'adp_by_pos'] <= 6:
            if df.loc[i,'pos_rank'] <= 6:
                df.loc[i,'success'] = 1
            else:
                df.loc[i, 'success'] = 0

        elif df.loc[i, 'adp_by_pos'] > 36:
            if df.loc[i,'pos_rank'] < 36:
                df.loc[i,'success'] = 1
            else:
                df.loc[i, 'success'] = 0

        else:
            if df.loc[i,'pos_rank'] <= df.loc[i,'adp_by_pos']:
                df.loc[i, 'success'] = 1
            else:
                df.loc[i, 'success'] = 0
        
        
for i in range(0,len(df.index)):
    if df.loc[i,'avg_draft_pos_ppr'] <= 12:
        df.loc[i, 'round'] = 1
    elif df.loc[i,'avg_draft_pos_ppr'] <= 24:
        df.loc[i, 'round'] = 2
    elif df.loc[i,'avg_draft_pos_ppr'] <= 36:
        df.loc[i, 'round'] = 3
    elif df.loc[i,'avg_draft_pos_ppr'] <= 48:
        df.loc[i, 'round'] = 4
    elif df.loc[i,'avg_draft_pos_ppr'] <= 60:
        df.loc[i, 'round'] = 5
    elif df.loc[i,'avg_draft_pos_ppr'] <= 72:
        df.loc[i, 'round'] = 6
    elif df.loc[i,'avg_draft_pos_ppr'] <= 84:
        df.loc[i, 'round'] = 7
    elif df.loc[i,'avg_draft_pos_ppr'] <= 96:
        df.loc[i, 'round'] = 8
    elif df.loc[i,'avg_draft_pos_ppr'] <= 108:
        df.loc[i, 'round'] = 9
    elif df.loc[i,'avg_draft_pos_ppr'] <= 120:
        df.loc[i, 'round'] = 10
    elif df.loc[i,'avg_draft_pos_ppr'] <= 132:
        df.loc[i, 'round'] = 11
    elif df.loc[i,'avg_draft_pos_ppr'] <= 144:
        df.loc[i, 'round'] = 12
    elif df.loc[i,'avg_draft_pos_ppr'] <= 156:
        df.loc[i, 'round'] = 13
    elif df.loc[i,'avg_draft_pos_ppr'] <= 168:
        df.loc[i, 'round'] = 14
    else:
        df.loc[i, 'round'] = 15

In [333]:
qb_df = df[df['pos']=='QB']
rb_df = df[df['pos']=='RB']
wr_df = df[df['pos']=='WR']
te_df = df[df['pos']=='TE']

In [449]:
qb_df = qb_df[qb_df['player']!='Tim Boyle']

In [334]:
qb_df.drop(columns=['rk','pos','tgt','rec','rec_yards','y/r','rec_tds','standard_pts','vbd', 'team'], inplace=True)

In [335]:
def add_target(group):
    group['target'] = group['ppr_pts'].shift(-1)
    return group

In [336]:
qb_df = add_target(qb_df)
qb_df.head()

,player,age,g,gs,cmp,pass_att,pass_yds,pass_tds,int,rush_att,rush_yard,y/a,rush_tds,fmb,fl,rush_rec_tds,ppr_pts,pos_rank,year,avg_draft_pos,avg_draft_pos_ppr,adp_by_pos,success,round,target
0,Aaron Rodgers,33,16,16,401.0,610.0,4428.0,40.0,7.0,67.0,369.0,5.51,4.0,8.0,4.0,4,380.0,1,2016,32.3,38.7,2,1.0,4.0,347.5
1,Matt Ryan,31,16,16,373.0,534.0,4944.0,38.0,7.0,35.0,117.0,3.34,0.0,4.0,2.0,0,347.5,2,2016,183.0,146.5,20,1.0,13.0,332.3
2,Drew Brees,37,16,16,471.0,673.0,5208.0,37.0,15.0,23.0,20.0,0.87,2.0,5.0,4.0,2,332.3,3,2016,57.2,61.1,5,1.0,6.0,307.7
3,Andrew Luck,27,15,15,346.0,545.0,4240.0,31.0,13.0,64.0,341.0,5.33,2.0,6.0,5.0,2,307.7,4,2016,54.8,58.3,4,1.0,5.0,300.3
4,Kirk Cousins,28,16,16,406.0,606.0,4917.0,25.0,12.0,34.0,96.0,2.82,4.0,9.0,3.0,4,300.3,5,2016,110.4,109.6,13,1.0,10.0,286.9


In [337]:
inverse_cols = ['int','fmb','fl','pos_rank','avg_draft_pos','avg_draft_pos_ppr','adp_by_pos','round']

In [338]:
for col in inverse_cols:
    qb_df[col] *= -1

In [457]:
qb_cols = ['age','g','gs','cmp','pass_att','pass_yds','pass_tds','int','rush_att','rush_yard','y/a','rush_tds',
           'fmb','fl','rush_rec_tds','pos_rank','avg_draft_pos','avg_draft_pos_ppr','adp_by_pos','round','ppr_pts']

In [400]:
X_train = qb_df[qb_df['year']<=2019]
X_val = qb_df[(qb_df['year']==2020) | (qb_df['year']==2021)]
X_test = qb_df[qb_df['year']==2022]

In [401]:
y_train = X_train['target']
X_train.drop(columns='target', inplace = True)

y_val = X_val['target']
X_val.drop(columns='target', inplace = True)

y_test = X_test['target']
X_test.drop(columns='target', inplace = True)

In [402]:
X_train.drop(columns=['year'],inplace=True)

In [403]:
X_val.drop(columns=['year'], inplace=True)

In [404]:
X_test.drop(columns=['year'],inplace=True)

In [419]:
rs = RobustScaler()

In [406]:
qb_2022 = X_test

In [420]:
X_train[qb_cols] = rs.fit_transform(X_train[qb_cols])
X_val[qb_cols] = rs.transform(X_val[qb_cols])
X_test[qb_cols] = rs.transform(X_test[qb_cols])

In [421]:
qb_mod_cols = ['age','g','gs','cmp','pass_att','pass_yds','pass_tds','int','rush_att','rush_yard',
               'y/a','rush_tds','fmb','fl','rush_rec_tds','pos_rank','avg_draft_pos','avg_draft_pos_ppr',
               'adp_by_pos','round','ppr_pts']

In [422]:
xgb = xgboost.XGBRegressor(eval_metric='rmse',n_estimators=100, max_depth=4, 
                           subsample=.65, colsample_bytree=.8, seed=42,
                           eta=.15, gamma=100)

In [423]:
xgb.fit(X_train[qb_mod_cols], y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, early_stopping_rounds=None,
             enable_categorical=False, eta=0.15, eval_metric='rmse',
             feature_types=None, gamma=100, gpu_id=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=4,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, predictor=None, ...)

In [424]:
xgb_preds = xgb.predict(X_train[qb_mod_cols])

In [425]:
preds = pd.DataFrame({'actual':y_train,
                      'baseline':y_train.mean(),
                      'xgb_preds':xgb_preds})

In [426]:
baseline_rmse = rmse(preds, 'baseline')
baseline_rmse

117.64961605619176

In [427]:
xgb_rmse = rmse(preds, 'xgb_preds')
xgb_rmse

3.8222217386621944

In [428]:
xgb_val_preds = xgb.predict(X_val[qb_mod_cols])

In [429]:
val_preds = pd.DataFrame({'actual':y_val,
                          'baseline':y_train.mean(),
                          'xgb_val_preds':xgb_val_preds})

In [430]:
xgb_val_rmse = rmse(val_preds, 'xgb_val_preds')
xgb_val_rmse

39.502073010315875

In [503]:
def xgb_modeling(df, cols):
    
    X_train = df[df['year']<2021]
    X_val = df[df['year']<2022]
    X_test = df[df['year']==2022]
    
    y_train = X_train['target']
    X_train.drop(columns='target', inplace = True)
    
    y_val = X_val['target']
    X_val.drop(columns='target', inplace = True)

    X_test.drop(columns='target', inplace = True)
    
    X_train.drop(columns=['year'],inplace=True)
    X_val.drop(columns=['year'],inplace=True)
    X_test.drop(columns=['year'],inplace=True)
    
    ss = StandardScaler()
    
    X_train[cols] = ss.fit_transform(X_train[cols])
    X_val[cols] = ss.transform(X_val[cols])
    X_test[cols] = ss.transform(X_test[cols])
    
    cols.append('success')
    
    xgb = xgboost.XGBRegressor(eval_metric='rmse',n_estimators=100, max_depth=4, 
                               subsample=.65, colsample_bytree=.8, seed=42,
                               eta=.15, gamma=100)
    
    xgb.fit(X_train[cols], y_train)
    xgb_preds = xgb.predict(X_train[cols])
    
    preds = pd.DataFrame({'actual':y_train,
                          'baseline':y_train.mean(),
                          'xgb_preds':xgb_preds})
    
    xgb_val_preds = xgb.predict(X_val[cols])
    
    val_preds = pd.DataFrame({'actual':y_val,
                              'baseline':y_train.mean(),
                              'xgb_val_preds':xgb_val_preds})
    
    
    pos_2022 = pd.DataFrame({'player':X_test['player'],
                             'preds':xgb.predict(X_test[cols])})
    
    return preds, val_preds, pos_2022

In [505]:
qb_cols = ['age','g','gs','cmp','pass_att','pass_yds','pass_tds','int','rush_att','rush_yard','y/a','rush_tds',
           'fmb','fl','rush_rec_tds','pos_rank','avg_draft_pos','avg_draft_pos_ppr','adp_by_pos','round','ppr_pts']

In [464]:
qb_inverse_cols = ['int','fmb','fl','pos_rank','avg_draft_pos','avg_draft_pos_ppr','adp_by_pos','round']

In [467]:
for col in qb_inverse_cols:
    qb_df[col] = qb_df[col] * -1

In [468]:
qb_df

,player,age,g,gs,cmp,pass_att,pass_yds,pass_tds,int,rush_att,rush_yard,y/a,rush_tds,fmb,fl,rush_rec_tds,ppr_pts,pos_rank,year,avg_draft_pos,avg_draft_pos_ppr,adp_by_pos,success,round,target
0,Aaron Rodgers,33,16,16,401.0,610.0,4428.0,40.0,-7.0,67.0,369.0,5.51,4.0,-8.0,-4.0,4,380.0,-1,2016,-32.3,-38.7,-2,1.0,-4.0,347.5
1,Matt Ryan,31,16,16,373.0,534.0,4944.0,38.0,-7.0,35.0,117.0,3.34,0.0,-4.0,-2.0,0,347.5,-2,2016,-183.0,-146.5,-20,1.0,-13.0,332.3
2,Drew Brees,37,16,16,471.0,673.0,5208.0,37.0,-15.0,23.0,20.0,0.87,2.0,-5.0,-4.0,2,332.3,-3,2016,-57.2,-61.1,-5,1.0,-6.0,307.7
3,Andrew Luck,27,15,15,346.0,545.0,4240.0,31.0,-13.0,64.0,341.0,5.33,2.0,-6.0,-5.0,2,307.7,-4,2016,-54.8,-58.3,-4,1.0,-5.0,300.3
4,Kirk Cousins,28,16,16,406.0,606.0,4917.0,25.0,-12.0,34.0,96.0,2.82,4.0,-9.0,-3.0,4,300.3,-5,2016,-110.4,-109.6,-13,1.0,-10.0,286.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3134,Jacob Eason,25,1,0,3.0,5.0,59.0,0.0,-1.0,0.0,0.0,0.00,0.0,-0.0,-0.0,0,0.4,-74,2022,-626.0,-641.0,-68,0.0,-15.0,0.3
3135,Case Keenum,34,2,0,2.0,7.0,8.0,0.0,-0.0,5.0,0.0,0.00,0.0,-0.0,-0.0,0,0.3,-75,2022,-457.0,-514.0,-43,0.0,-15.0,-0.4
3136,Nate Sudfeld,29,2,0,0.0,0.0,0.0,0.0,-0.0,5.0,-4.0,-0.80,0.0,-0.0,-0.0,0,-0.4,-76,2022,-536.0,-566.0,-63,0.0,-15.0,-0.5
3137,Chad Henne,37,3,0,0.0,2.0,0.0,0.0,-0.0,5.0,-5.0,-1.00,0.0,-1.0,-0.0,0,-0.5,-77,2022,-475.0,-530.0,-47,0.0,-15.0,-1.0


In [506]:
qb_preds, qb_val_preds, qb_2022 = xgb_modeling(qb_df, qb_cols)

In [532]:
qb_2022.sort_values('preds',ascending=False).head(20)

,player,preds
3062,Josh Allen,375.828430
3063,Jalen Hurts,369.966736
3061,Patrick Mahomes,356.720337
3064,Joe Burrow,337.917511
3067,Trevor Lawrence,289.225861
3065,Geno Smith,286.852020
3068,Kirk Cousins,285.712830
3070,Jared Goff,284.814117
3069,Daniel Jones,282.683655
3066,Justin Fields,271.617126


In [507]:
rmse(qb_preds,'baseline')

120.31155444270327

In [508]:
rmse(qb_preds, 'xgb_preds')

3.8219950996686913

In [509]:
rmse(qb_val_preds, 'xgb_val_preds')

20.672813417651003

In [364]:
rb_df.head()

,player,rk,team,pos,age,g,gs,cmp,pass_att,pass_yds,pass_tds,int,rush_att,rush_yard,y/a,rush_tds,tgt,rec,rec_yards,y/r,rec_tds,fmb,fl,rush_rec_tds,standard_pts,ppr_pts,vbd,pos_rank,year,avg_draft_pos,avg_draft_pos_ppr,adp_by_pos,success,round
66,David Johnson,1,ARI,RB,25,16,16,0.0,0.0,0.0,0.0,0.0,293.0,1239.0,4.23,16.0,120.0,80.0,879.0,10.99,4.0,5.0,3.0,20,328.0,407.8,191.0,1,2016,5.4,3.4,1,1.0,1.0
67,Ezekiel Elliott,2,DAL,RB,21,15,15,0.0,0.0,0.0,0.0,0.0,322.0,1631.0,5.07,15.0,39.0,32.0,363.0,11.34,1.0,5.0,1.0,16,293.0,325.4,156.0,2,2016,6.3,8.3,3,1.0,1.0
68,Le'Veon Bell,5,PIT,RB,24,12,12,0.0,1.0,0.0,0.0,0.0,261.0,1268.0,4.86,7.0,94.0,75.0,616.0,8.21,2.0,4.0,1.0,9,242.0,317.4,105.0,3,2016,10.1,12.3,6,1.0,2.0
69,LeSean McCoy,4,BUF,RB,28,15,15,0.0,0.0,0.0,0.0,0.0,234.0,1267.0,5.41,13.0,57.0,50.0,356.0,7.12,1.0,3.0,0.0,14,248.0,298.3,111.0,4,2016,19.7,22.0,9,1.0,2.0
70,DeMarco Murray,6,TEN,RB,28,16,16,1.0,2.0,10.0,1.0,0.0,293.0,1287.0,4.39,9.0,67.0,53.0,377.0,7.11,3.0,3.0,1.0,12,241.0,293.8,104.0,5,2016,36.5,40.3,14,1.0,4.0


In [473]:
rb_df.drop(columns=['rk','team','pos','cmp','pass_att','pass_yds','pass_tds','int','standard_pts','vbd'], inplace=True)

In [365]:
rb_df = add_target(rb_df)

In [513]:
rb_cols = ['age','g','gs','rush_att','rush_yard','y/a','rush_tds','tgt','rec','rec_yards','y/r','rec_tds','fmb',
           'fl','rush_rec_tds','ppr_pts','pos_rank','avg_draft_pos','avg_draft_pos_ppr','adp_by_pos','round']

In [480]:
rb_inverse_cols = ['fmb','fl','pos_rank','avg_draft_pos','avg_draft_pos_ppr','adp_by_pos','round']

In [483]:
for col in rb_inverse_cols:
    rb_df[col] = rb_df[col]*-1

In [510]:
rb_df

,player,age,g,gs,rush_att,rush_yard,y/a,rush_tds,tgt,rec,rec_yards,y/r,rec_tds,fmb,fl,rush_rec_tds,ppr_pts,pos_rank,year,avg_draft_pos,avg_draft_pos_ppr,adp_by_pos,success,round,target
66,David Johnson,25,16,16,293.0,1239.0,4.23,16.0,120.0,80.0,879.0,10.99,4.0,-5.0,-3.0,20,407.8,-1,2016,-5.4,-3.4,-1,1.0,-1.0,325.4
67,Ezekiel Elliott,21,15,15,322.0,1631.0,5.07,15.0,39.0,32.0,363.0,11.34,1.0,-5.0,-1.0,16,325.4,-2,2016,-6.3,-8.3,-3,1.0,-1.0,317.4
68,Le'Veon Bell,24,12,12,261.0,1268.0,4.86,7.0,94.0,75.0,616.0,8.21,2.0,-4.0,-1.0,9,317.4,-3,2016,-10.1,-12.3,-6,1.0,-2.0,298.3
69,LeSean McCoy,28,15,15,234.0,1267.0,5.41,13.0,57.0,50.0,356.0,7.12,1.0,-3.0,-0.0,14,298.3,-4,2016,-19.7,-22.0,-9,1.0,-2.0,293.8
70,DeMarco Murray,28,16,16,293.0,1287.0,4.39,9.0,67.0,53.0,377.0,7.11,3.0,-3.0,-1.0,12,293.8,-5,2016,-36.5,-40.3,-14,1.0,-4.0,284.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3281,Reggie Bonnafon,26,1,0,3.0,8.0,2.67,0.0,0.0,0.0,0.0,0.00,0.0,-0.0,-0.0,0,0.8,-142,2022,-668.0,-683.0,-136,0.0,-15.0,0.7
3282,Kylin Hill,24,2,0,1.0,7.0,7.00,0.0,0.0,0.0,0.0,0.00,0.0,-0.0,-0.0,0,0.7,-143,2022,-302.0,-302.0,-94,0.0,-15.0,0.4
3283,Duke Johnson,29,1,0,2.0,4.0,2.00,0.0,0.0,0.0,0.0,0.00,0.0,-0.0,-0.0,0,0.4,-144,2022,-277.0,-296.0,-92,0.0,-15.0,0.3
3284,Nick Bellore,33,16,0,1.0,3.0,3.00,0.0,0.0,0.0,0.0,0.00,0.0,-0.0,-0.0,0,0.3,-145,2022,-663.0,-678.0,-134,0.0,-15.0,0.2


In [514]:
rb_preds, rb_val_preds, rb_2022 = xgb_modeling(rb_df, rb_cols)

In [515]:
rmse(rb_preds,'baseline')

84.97482573685176

In [516]:
rmse(rb_preds,'xgb_preds')

2.7871360164100047

In [517]:
rmse(rb_val_preds,'xgb_val_preds')

14.182357114650848

In [533]:
rb_2022.sort_values('preds',ascending=False).head(20)

,player,preds
3140,Austin Ekeler,348.460724
3141,Christian McCaffrey,324.322479
3142,Josh Jacobs,293.031494
3143,Derrick Henry,290.152405
3144,Saquon Barkley,273.064606
3145,Nick Chubb,269.421936
3285,Damien Williams,253.915604
3146,Rhamondre Stevenson,241.301407
3148,Aaron Jones,241.022125
3147,Tony Pollard,240.128723


In [373]:
wr_df.drop(columns=['rk','team','pos','cmp','pass_tds','pass_att','pass_yds','int','rush_att','rush_yard','y/a','rush_tds',
                    'standard_pts','vbd'],inplace=True)

In [374]:
wr_df = add_target(wr_df)

In [375]:
wr_df.head()

,player,age,g,gs,tgt,rec,rec_yards,y/r,rec_tds,fmb,fl,rush_rec_tds,ppr_pts,pos_rank,year,avg_draft_pos,avg_draft_pos_ppr,adp_by_pos,success,round,target
208,Antonio Brown,28,15,15,154.0,106.0,1284.0,12.11,12.0,0.0,0.0,12,307.3,1,2016,1.3,1.2,1,1.0,1.0,304.7
209,Jordy Nelson,31,16,16,152.0,97.0,1257.0,12.96,14.0,1.0,1.0,14,304.7,2,2016,19.3,17.3,11,1.0,2.0,304.1
210,Mike Evans,23,16,16,173.0,96.0,1321.0,13.76,12.0,0.0,0.0,12,304.1,3,2016,15.2,15.7,9,1.0,2.0,296.6
211,Odell Beckham Jr.,24,16,16,169.0,101.0,1367.0,13.53,10.0,3.0,1.0,10,296.6,4,2016,4.0,3.4,3,1.0,1.0,273.8
212,T.Y. Hilton,27,16,16,155.0,91.0,1448.0,15.91,6.0,0.0,0.0,6,273.8,5,2016,31.5,29.4,17,1.0,3.0,259.9


In [519]:
wr_cols = ['age','g','gs','tgt','rec','rec_yards','y/r','rec_tds','fmb','fl','rush_rec_tds','ppr_pts','pos_rank',
           'avg_draft_pos','avg_draft_pos_ppr','adp_by_pos','round']

In [ ]:
wr_inverse_cols = ['fmb','fl','pos_rank','avg_draft_pos','avg_draft_pos_ppr','adp_by_pos','round']

In [492]:
for col in wr_inverse_cols:
    wr_df[col] = wr_df[col] * -1

In [520]:
wr_preds, wr_val_preds, wr_2022 = xgb_modeling(wr_df, wr_cols)

In [521]:
rmse(wr_preds, 'baseline')

81.9147791528008

In [522]:
rmse(wr_preds,'xgb_preds')

1.957026904370617

In [523]:
rmse(wr_val_preds,'xgb_val_preds')

3.3881930423367956

In [534]:
wr_2022.sort_values('preds',ascending=False).head(20)

,player,preds
3287,Tyreek Hill,326.390198
3289,Stefon Diggs,316.016724
3288,Davante Adams,304.916016
3290,CeeDee Lamb,297.281586
3291,A.J. Brown,288.919678
3286,Justin Jefferson,280.402008
3292,Amon-Ra St. Brown,259.334900
3293,Jaylen Waddle,254.042053
3294,DeVonta Smith,246.973648
3295,Amari Cooper,239.578323


In [382]:
te_df.drop(columns=['rk','team','pos','cmp','pass_att','pass_yds','pass_tds','int','rush_att','rush_yard','y/a',
                    'rush_tds','standard_pts','vbd'], inplace=True)

In [383]:
te_df = add_target(te_df)

In [384]:
te_df.head()

,player,age,g,gs,tgt,rec,rec_yards,y/r,rec_tds,fmb,fl,rush_rec_tds,ppr_pts,pos_rank,year,avg_draft_pos,avg_draft_pos_ppr,adp_by_pos,success,round,target
379,Travis Kelce,27,16,15,117.0,85.0,1125.0,13.24,4.0,0.0,0.0,4,223.0,1,2016,61.4,64.3,6,1.0,6.0,209.0
380,Kyle Rudolph,27,16,16,132.0,83.0,840.0,10.12,7.0,0.0,0.0,7,209.0,2,2016,339.0,322.0,26,1.0,15.0,207.3
381,Greg Olsen,31,16,16,129.0,80.0,1073.0,13.41,3.0,0.0,0.0,3,207.3,3,2016,51.3,48.4,4,1.0,5.0,189.3
382,Jimmy Graham,30,16,15,95.0,65.0,923.0,14.20,6.0,2.0,2.0,6,189.3,4,2016,124.0,144.3,16,1.0,13.0,188.1
383,Delanie Walker,32,15,9,102.0,65.0,800.0,12.31,7.0,1.0,0.0,7,188.1,5,2016,68.9,64.0,5,1.0,6.0,183.6


In [525]:
te_cols = ['age','g','gs','tgt','rec','rec_yards','y/r','rec_tds','fmb','fl','rush_rec_tds','ppr_pts','pos_rank',
           'avg_draft_pos','avg_draft_pos_ppr','adp_by_pos','round']

In [497]:
te_inverse_cols = ['fmb','fl','pos_rank','avg_draft_pos','avg_draft_pos_ppr','adp_by_pos','round']

In [498]:
for col in te_inverse_cols:
    te_df[col] = te_df[col] * -1

In [526]:
te_preds, te_val_preds, te_2022 = xgb_modeling(te_df, te_cols)

In [527]:
rmse(te_preds, 'baseline')

58.819862313810674

In [528]:
rmse(te_preds, 'xgb_preds')

2.5776438733778475

In [529]:
rmse(te_val_preds,'xgb_val_preds')

10.833581900987786

In [535]:
te_2022.sort_values('preds',ascending=False).head(20)

,player,preds
3477,Travis Kelce,276.611633
3478,T.J. Hockenson,205.830307
3479,George Kittle,190.576050
3480,Mark Andrews,186.949661
3481,Evan Engram,167.918350
3482,Tyler Higbee,149.692810
3483,Pat Freiermuth,149.201202
3484,Cole Kmet,147.749756
3585,Feleipe Franks,147.270218
3485,Taysom Hill,142.489960
